<a href="https://colab.research.google.com/github/HyunLee103/Audio_data_preprocessing/blob/master/Music_genre_classification_RNN(LSTM).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import json
import numpy as np
from sklearn.model_selection import train_test_split
import tensorflow.keras as keras
import matplotlib.pyplot as plt

In [0]:
# only in colab
%tensorflow_version 2.x

import tensorflow as tf

In [0]:
DATA_PATH = "/content/drive/My Drive/Sound AI/data_10.json"

In [0]:
# 데이터 불러오기

def load_data(data_path):
  with open(data_path, "r") as fp :
    data = json.load(fp)

  X = np.array(data['mfcc'])
  y = np.array(data['labels'])

  return X,y

In [0]:
# history 함수 

def plot_history(history):

  fig, axs = plt.subplot(2)

  axs[0].plot(history.history['accuaracy'], label = 'train acc')
  axs[0].plot(history.history['val_accuracy'], label = 'val acc')
  
  axs[1].plot(history.history['loss'],label='train loss')
  axs[1].plot(history.history['val_loss'], label='val loss')

  plt.show()

In [0]:
# 데이터 전처리 

def prepare_data(test_size, val_size):
  X,y = load_data(DATA_PATH)

  X_train, X_test, y_train, y_test = train_test_split(X, y, test_size= test_size)
  X_train, X_val , y_train, y_val = train_test_split(X_train,y_train, test_size=val_size)

  return X_train, X_val, X_test, y_train, y_val, y_test

In [0]:
# 모델 

def modeling(input):
  model = tf.keras.Sequential()

  model.add(tf.keras.layers.LSTM(128,input_shape = input, return_sequences=True)) # return_sequences = True 는 아웃풋을 sequences로 출력하겠다는 뜻
                                                                                 # 이걸 해야 이 sequence 아웃풋을 다음 LSTM layer에 넣을 수 있다.
  model.add(tf.keras.layers.LSTM(128, return_sequences=True))                     # RNN 계열은 모델안에 activation 함수가 들어가 있다. 따로 줄 필요 없음
  model.add(tf.keras.layers.LSTM(128, return_sequences=False))

  model.add(tf.keras.layers.Dense(128,activation='relu'))
  model.add(tf.keras.layers.Dropout(0.3))

  model.add(tf.keras.layers.Dense(10,activation='softmax'))

  return model 

# Q : RNN 계열에서 배치놈 안하나?
# A : 가능은 한데 CNN, DNN 계열에 비해 의미가 없다. 왜냐면 CNN에서는 BN이 채널 차원으로 적용된다. RNN에서도 동일하게 수직방향으로 BN이 작용해
# BN이 미니배치와 각 step에 대해 적용된다. 따라서 RNN에서 중요하게 여기는 시계열성을 반영하지 못하므로 그 효과가 미비하다.
# 그래서 layer normalization이라는 새로운 기법이 제시되었다.


In [0]:
if __name__ == "__main__":

    # get train, validation, test splits
    X_train, X_validation, X_test, y_train, y_validation, y_test = prepare_data(0.25, 0.2)

    # create network
    input_shape = (X_train.shape[1], X_train.shape[2]) # 130, 13
    model = modeling(input_shape)

    # compile model
    optimiser = keras.optimizers.Adam(learning_rate=0.0001)
    model.compile(optimizer=optimiser,
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])

    model.summary()

    # train model
    history = model.fit(X_train, y_train, validation_data=(X_validation, y_validation), batch_size=32, epochs=30)

    # plot accuracy/error for training and validation
    plot_history(history)

    # evaluate model on test set
    test_loss, test_acc = model.evaluate(X_test, y_test, verbose=2)
    print('\nTest accuracy:', test_acc)

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 259, 128)          72704     
_________________________________________________________________
lstm_1 (LSTM)                (None, 259, 128)          131584    
_________________________________________________________________
lstm_2 (LSTM)                (None, 128)               131584    
_________________________________________________________________
dense (Dense)                (None, 128)               16512     
_________________________________________________________________
dropout (Dropout)            (None, 128)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 10)                1290      
Total params: 353,674
Trainable params: 353,674
Non-trainable params: 0
__________________________________________________

ValueError: ignored

<Figure size 432x288 with 0 Axes>